### Problem
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Build Spark Session

In [ ]:
spark = SparkSession.builder.getOrCreate()
import findspark
findspark.init()
from pyspark.sql import SparkSession

## Data Loading


Read two datasets: 
* Train
* Test.



In [ ]:
train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test.csv', header=True, inferSchema=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [ ]:
print(type(train_df))
print(type(test_df))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


**Show 5 rows.**

In [ ]:
train_df.show(5, truncate=False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

**Display schema for the dataset:**

In [ ]:
train_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [ ]:
train_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [ ]:
from pyspark.sql.functions import *
import pyspark.sql.functions as fns
train_count = train_df.count()
train_count

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [ ]:
Survived = train_df.filter('Survived = 1').count()
notSurvived = train_df.filter('Survived = 0').count()

**Display your result:**

In [ ]:
print(f'Survived {Survived}')
print(f'Not Survived {notSurvived}')

Survived 342
Not Survived 549


**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [ ]:
print(f'Survived ratio {Survived/train_count}')
print(f'Not Survived ratio {notSurvived/train_count}')

Survived ratio 0.3838383838383838
Not Survived ratio 0.6161616161616161


**Can you get the number of males and females?**


In [ ]:
male = train_df.filter('sex = "male"').count()
female = train_df.filter('sex = "female"').count()
male, female

(577, 314)

**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [ ]:
train_df.select('sex', 'Survived').groupBy('sex').mean('Survived').show()

+------+-------------------+
|   sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Create temporary view PySpark:**

In [ ]:
train_df.createOrReplaceTempView("view")

**How many people survived, and how many didn't survive? By SQL:**

In [ ]:
spark.sql('SELECT SUM(Survived) AS Survived, (COUNT(*)-SUM(Survived)) AS notSurvived from view').show()

+--------+-----------+
|Survived|notSurvived|
+--------+-----------+
|     342|        549|
+--------+-----------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [ ]:
spark.sql("""
        SELECT sex, SUM(Survived)/COUNT(Survived)
        FROM view
        GROUP BY sex
""").show()

+------+---------------------------------------------------------------------------------+
|   sex|(CAST(sum(CAST(Survived AS BIGINT)) AS DOUBLE) / CAST(count(Survived) AS DOUBLE))|
+------+---------------------------------------------------------------------------------+
|female|                                                               0.7420382165605095|
|  male|                                                              0.18890814558058924|
+------+---------------------------------------------------------------------------------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [ ]:
spark.sql("""
        SELECT Pclass, SUM(Survived)/COUNT(Survived)
        FROM view
        GROUP BY Pclass
""").show()

+------+---------------------------------------------------------------------------------+
|Pclass|(CAST(sum(CAST(Survived AS BIGINT)) AS DOUBLE) / CAST(count(Survived) AS DOUBLE))|
+------+---------------------------------------------------------------------------------+
|     1|                                                               0.6296296296296297|
|     3|                                                              0.24236252545824846|
|     2|                                                              0.47282608695652173|
+------+---------------------------------------------------------------------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [ ]:
all_df = train_df.union(test_df)

**Display count:**

In [ ]:
all_df.count()

1329

**Can you define the number of null values in each column?**


In [ ]:
all_df.agg(*[count(when(isnull(c), c)).alias(c) for c in all_df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [ ]:
null_df = all_df.agg(*[count(when(isnull(c), c)).alias(c) for c in all_df.columns])
null_df.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Preprocessing 

**Create Temporary view PySpark:**

In [ ]:
all_df.createOrReplaceTempView('myView')

**Can you show the "name" column from your temporary table?**

In [ ]:
spark.sql('SELECT name FROM myView').show(truncate=False)

+-------------------------------------------------------+
|name                                                   |
+-------------------------------------------------------+
|Braund, Mr. Owen Harris                                |
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |
|Heikkinen, Miss. Laina                                 |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)           |
|Allen, Mr. William Henry                               |
|Moran, Mr. James                                       |
|McCarthy, Mr. Timothy J                                |
|Palsson, Master. Gosta Leonard                         |
|Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)      |
|Nasser, Mrs. Nicholas (Adele Achem)                    |
|Sandstrom, Miss. Marguerite Rut                        |
|Bonnell, Miss. Elizabeth                               |
|Saundercock, Mr. William Henry                         |
|Andersson, Mr. Anders Johan                            |
|Vestrom, Miss

**Run this code:**

In [ ]:
import pyspark.sql.functions as F
all_df = all_df.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
all_df.createOrReplaceTempView('combined')

**Display "Title" column and count "Title" column:**

In [ ]:
spark.sql('SELECT Title, COUNT(*) FROM combined GROUP BY Title').show()

+--------+--------+
|   Title|count(1)|
+--------+--------+
|     Don|       1|
|    Miss|     257|
|Countess|       2|
|     Col|       4|
|     Rev|       9|
|    Lady|       2|
|  Master|      56|
|     Mme|       1|
|    Capt|       2|
|      Mr|     786|
|      Dr|      11|
|     Mrs|     186|
|     Sir|       2|
|Jonkheer|       2|
|    Mlle|       4|
|   Major|       3|
|      Ms|       1|
+--------+--------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [ ]:
titles_map = {'Dr':'rare', 'Rev':'rare', 'Major':'rare', 'Col':'rare', 
              'Mlle':'rare', 'Capt':'rare', 'Don':'rare', 
              'Jonkheer':'rare', 'Countess':'rare', 'Ms':'rare', 
              'Sir':'rare', 'Lady':'rare', 'Mme':'rare'}

**Run the function:**

In [ ]:
def impute_title(title):
    if title in titles_map:
        return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.
    return title

**Apply the function on "Title" column using UDF:**

In [ ]:
from pyspark.sql.types import *

impute_titleUDF = udf(lambda z: impute_title(z), StringType())

In [ ]:
all_df = all_df.withColumn('Title', impute_titleUDF(all_df['Title']))

**Display "Title" from table and group by "Title" column:**

In [ ]:
all_df.select('Title').groupBy('Title').count().show()

+------+-----+
| Title|count|
+------+-----+
|  rare|   44|
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
+------+-----+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [ ]:
all_df.agg(mean('age')).collect()[0][0]

30.079501879699244

**Fill missing with "age" mean:**

In [ ]:
all_df = all_df.na.fill(all_df.agg(mean('age')).collect()[0][0], subset=['age'])

## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [ ]:
grouped_Embarked = all_df.select('Embarked')\
                         .groupBy('Embarked')\
                         .count()\
                         .sort('Embarked',ascending=False)

**Show "groupped_Embarked" your variable:**

In [ ]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       Q|  111|
|       C|  253|
|    null|    3|
+--------+-----+



**Get max of groupped_Embarked:** 

In [ ]:
max_embarked = grouped_Embarked.collect()[0][0]
max_embarked

'S'

**Fill missing values with max 'S' of grouped_Embarked:**

In [ ]:
all_df = all_df.na.fill(max_embarked, subset=['Embarked'])

In [ ]:
all_df.select('Embarked')\
      .groupBy('Embarked')\
      .count()\
      .sort('Embarked',ascending=False).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  965|
|       Q|  111|
|       C|  253|
+--------+-----+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [ ]:
import numpy as np
def firstChar(cabin):
    if type(cabin) == str:
        return cabin[0]
    return cabin

In [ ]:
firstCharUDF = udf(lambda z: firstChar(z), StringType())

In [ ]:
all_df = all_df.withColumn('cabin', firstCharUDF(all_df['cabin']))

**Show the result:**

In [ ]:
all_df.select('cabin').show()

+-----+
|cabin|
+-----+
| null|
|    C|
| null|
|    C|
| null|
| null|
|    E|
| null|
| null|
| null|
|    G|
|    C|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 20 rows



**Create the temporary view:**

In [ ]:
all_df.createOrReplaceTempView('df_view')

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [ ]:
spark.sql("""
        SELECT cabin, COUNT(*) as count 
        FROM df_view 
        GROUP BY cabin 
        ORDER BY count
""").show()

+-----+-----+
|cabin|count|
+-----+-----+
|    T|    1|
|    G|    4|
|    F|   18|
|    A|   23|
|    E|   51|
|    D|   52|
|    B|   77|
|    C|   82|
| null| 1021|
+-----+-----+



**Fill missing values with "U":**

In [ ]:
all_df = all_df.na.fill('U', subset=['cabin'])

In [ ]:
all_df.select('cabin').groupBy('cabin').count().show()

+-----+-----+
|cabin|count|
+-----+-----+
|    F|   18|
|    E|   51|
|    T|    1|
|    B|   77|
|    U| 1021|
|    D|   52|
|    C|   82|
|    A|   23|
|    G|    4|
+-----+-----+



**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categoricalCols = [field for (field, dataType) in all_df.dtypes
                   if dataType == "string"]

indexOutputCols = [x + "_Index" for x in categoricalCols]



stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')

**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [ ]:
oheOutputCols = [x + "_OHE" for x in categoricalCols]

oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

numericCols = [field for (field,dataType) in all_df.dtypes
              if ((dataType!='string')& (field!='Survived'))]

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [ ]:
from pyspark.ml.feature import VectorAssembler

assemblerInputs = oheOutputCols + numericCols

vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [ ]:
trainDF, testDF = all_df.randomSplit([.8,.2],seed=42)

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier 

rf = RandomForestClassifier(featuresCol='features',labelCol='Survived')

In [ ]:
from pyspark.ml import Pipeline

pipeline =Pipeline(stages = [stringIndexer,oheEncoder,vecAssembler,rf])

pipelineModel = pipeline.fit(trainDF)
predDF = pipelineModel.transform(testDF)

In [ ]:
predictions = pipeline.fit(trainDF).transform(testDF)

predictions.select("prediction", "Survived", "features").show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|(1464,[600,811,11...|
|       1.0|       1|(1464,[533,1414,1...|
|       0.0|       0|(1464,[684,811,87...|
|       0.0|       1|(1464,[515,1225,1...|
|       0.0|       1|(1464,[366,811,11...|
|       0.0|       1|(1464,[288,811,13...|
|       1.0|       1|(1464,[444,878,14...|
|       0.0|       0|(1464,[556,1341,1...|
|       1.0|       1|(1464,[582,829,14...|
|       1.0|       1|(1464,[679,865,14...|
|       0.0|       0|(1464,[786,811,14...|
|       0.0|       0|(1464,[407,811,12...|
|       0.0|       0|(1464,[702,811,84...|
|       1.0|       1|(1464,[283,1156,1...|
|       0.0|       0|(1464,[385,811,11...|
|       0.0|       0|(1464,[474,811,87...|
|       0.0|       1|(1464,[643,1276,1...|
|       0.0|       0|(1464,[693,811,82...|
|       1.0|       1|(1464,[336,1142,1...|
|       0.0|       1|(1464,[463,1172,1...|
+----------

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
print("Accuracy : " + str(evaluator.evaluate(predictions)))

Accuracy : 0.7972027972027972
